# Webscrape Bichitra Library

### Import this library as: `import Webscrape_Bichitra_Library as wbl`

Import necessary python packages.

In [51]:
import os
import errno
import requests
from bs4 import BeautifulSoup as bsp
import urllib.parse as urlparse
from numpy import arange
import time # Refer to: https://stackoverflow.com/questions/3620943/measuring-elapsed-time-with-the-time-module
import pandas as pd
import matplotlib.pyplot as plt
from numpy import random as np_random, floor as np_floor
%matplotlib inline

## Function: Determine Encoding from URL

In [52]:
def get_HTML_Encoding(target_url):
    source_html_2 = requests.get(target_url)
    return source_html_2.encoding

## Functions for Manuscript Transcript Access

In [53]:
# List of functions

def saveTextAsFile(writeThis, fileNamePath, filePermission = "w+", fileEncoding = 'utf-8'):    
    """
    To create a file object with various file-permission aparmeters use premission:
            r (read)
            w (write) ; w+ (write and "create-file if not present")
            a (append) ; a+ (append and "create-file if not present")

            Source: https://www.guru99.com/reading-and-writing-files-in-python.html
    """
    #fileName = "RBVBMS_009A.txt"
    makeDirWhenNotPresent(fileNamePath)
    with open(fileNamePath, filePermission, encoding = fileEncoding) as fw:
    #fw = open(fileNamePath, filePermission, encoding = fileEncoding)    
        fw.write(writeThis)
        fw.close()
    
def getFullPath(fileName, basePath = os.getcwd()):
    completePath = os.path.join(basePath,fileName)
    print("Complete FilePath: " + completePath)
    
    return completePath

def makeSubFolderPath(subFolderList = [""], rootDirLocation = os.getcwd()):
    #    import os
    if isinstance(subFolderList,list)==0:
        subFolderList = [str(subFolderList)]
    basePath = os.path.join(rootDirLocation,os.path.join(*subFolderList))
    print("RelativePath: " + basePath)
    
    return basePath  

def readFile(fileNamePath, fileEncoding = 'utf-8', iDebugFlag = 1):
    fr = open(fileNamePath, 'r',encoding = fileEncoding)
    text = fr.read();#.decode('utf8')
    if (iDebugFlag == 1):
        print(text)
        
    return text

def makeDirWhenNotPresent(fileNamePath):
    #    import os
    #    import errno
    if not os.path.exists(os.path.dirname(fileNamePath)):
        try:
            os.makedirs(os.path.dirname(fileNamePath))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    
    dirExistsStatus = os.path.exists(os.path.dirname(fileNamePath))
    
    return dirExistsStatus

def get_Transcript(target_url):
    # Imports required for this section
    #        import requests
    #        from bs4 import BeautifulSoup as bsp
    
    # Get source html text.
    source_html = requests.get(target_url).text

    # Use BeautifulSoup to extract intended content.
    soup = bsp(source_html, 'html.parser')
    transcript_html = soup.find("div", {"class": "transcript_con bn_content"})
    
    return transcript_html.get_text()


def example_GetTranscriptText():

    target_url = "http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=9&mname=RBVBMS_009A";
    #"https://rabindra-rachanabali.nltr.org/node/1763"; 
    #"https://en.wikipedia.org/wiki/Wikipedia"; 

    transcript_text = get_Transcript(target_url)
    print(transcript_text)
    
    return transcript_text



In [54]:
"""
# Example Codes:

def example_SaveTranscriptAsText_And_Read(transcript_text):
    fileName = "RBVBMS_009A.txt"
    writeThis = transcript_text;#.encode('utf8')); #.decode('utf8')
    basePath = makeSubFolderPath("Tasher_Desh")
    print(basePath)
    fileNamePath = getFullPath(fileName, basePath)
    print(fileNamePath)
    # Save transcript_text to File
    fileEncoding = "utf-8"
    saveTextAsFile(transcript_text, fileNamePath, "w+", fileEncoding)
    readFile(fileNamePath, "utf-8", 1)    
    
"""

'\n# Example Codes:\n\ndef example_SaveTranscriptAsText_And_Read(transcript_text):\n    fileName = "RBVBMS_009A.txt"\n    writeThis = transcript_text;#.encode(\'utf8\')); #.decode(\'utf8\')\n    basePath = makeSubFolderPath("Tasher_Desh")\n    print(basePath)\n    fileNamePath = getFullPath(fileName, basePath)\n    print(fileNamePath)\n    # Save transcript_text to File\n    fileEncoding = "utf-8"\n    saveTextAsFile(transcript_text, fileNamePath, "w+", fileEncoding)\n    readFile(fileNamePath, "utf-8", 1)    \n    \n'

# Download Manuscript Image Files

### Extract Image File URL  and Total Number of Images (pages) from Manuscript URL:

In [89]:
def getFromManuscriptURL_ImageURL_KeyValue(target_url, iDebugFlag = 0):

    # This is the index of the lines in the <script></script> 
    #            portion where the Image File URL KeyValue is
    #            available. 
    #      NOTE: If the design of the javascript 
    #            function: **change_page(pageno, inp_value)** 
    #            ever changes, this needs to be readjusted to 
    #            pull the KeyValue data from the relevant line.
    
    target_line_index = 19

    #target_url = target_url_list[0]
    # Get source html text.
    source_html = requests.get(target_url)
    if iDebugFlag == 1:
        print("\n Target HTML Text: \n" + source_html.text + "\n")

    # Use BeautifulSoup to extract intended content.
    soup_head = bsp(source_html.text, 'html.parser').find("head")
    # Create List of <script></script> tags' data
    soup_head_js = soup_head.findAll("script", {"type": "text/javascript"}) 
    # pick last java-script tag data from the list
    func_text = soup_head_js[-1:] 
    # Split into lines
    func_text_lines = str(func_text).splitlines()
    # Split the string in the target-line with: "=", "&img", "%" (in that order)
    fileKeyValue = str(func_text_lines[target_line_index])\
                        .split("=")[2].split(r'&img')[0]\
                        .split(r'%')[0]
    
    return fileKeyValue


def getFromManuscriptURL_TotalImagesNumber(target_url, iDebugFlag = 0):

    # Extract **total_pages_number** for a manuscript's images.
        
    #target_url = target_url_list[0]
    # Get source html text.
    source_html = requests.get(target_url)
    if iDebugFlag == 1:
        print("\n Target HTML Text: \n" + source_html.text + "\n")

    # Use BeautifulSoup to extract intended content.
    # Soup for HTML body
    soup_body = bsp(source_html.text, 'html.parser').find("body")
    # Soup for HTML body code containg target-area in the manuscript_toolbar
    soup_toolbar_manuscript = soup_body.find("div", {"class": "main_toolbar"})\
                                .find("div", {"class": "manuscript_toolbar"})\
                                .find("span", {"class": r"clearfix ui-widget ui-corner-all button_group"})
    # soup for <span></span> containing info: total-number-of-pages
    soup_page_number = soup_toolbar_manuscript.find("span", {"class": "man_button"})
    if iDebugFlag == 2:
        print("\n HTML Text for Total Number of Pages: \n" \
              + soup_page_number.prettify() + "\n")
    
    # Split into lines
    page_num_text_lines = str(soup_page_number.text).splitlines()
    # Pick last item from the list (page_num_text_lines) and 
    # split with "of " followed by stripping of white-spaces
    total_pages_number = int("".join("".join(page_num_text_lines[-1:])\
                                    .split("of ")[-1:]\
                                    ).strip()\
                            )
        
    return total_pages_number

In [91]:
"""
# Example

target_url = target_url_list[0]
image_file_url_keyValue = getFromManuscriptURL_ImageURL_KeyValue(target_url, 0)
print("Image File URL key-value: " + image_file_url_keyValue + "\n")

"""

'\n# Example\n\ntarget_url = target_url_list[0]\nimage_file_url_keyValue = getFromManuscriptURL_ImageURL_KeyValue(target_url, 0)\nprint("Image File URL key-value: " + image_file_url_keyValue + "\n")\n\n'

### Function: Check if the URL-Content is downloadable or not

In [57]:
def is_downloadable(target_url, content_length_max = 2e8):
    """
    Does the url contain a downloadable resource?
    Note: Set content_length_max = 0 for not using a content-size-limitation.
    Source: https://www.codementor.io/aviaryan/downloading-files-from-urls-in-python-77q3bs0un
    """
    # import requests
    # import pandas as pd
    h = requests.head(target_url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        print("content-type: text")
        return False
    if 'html' in content_type.lower():
        print("content-type: html")
        return False
    if 'image' in content_type.lower():
        print("content-type: " +str(content_type.lower()))
    # Ensure a specific min size 
    content_length = header.get('content-length', None)
    if not (content_length_max == 0):
        # content_length_max = 2e8 is about 200 mb (approx)
        if content_length and (content_length >= content_length_max):  
            print("content-length: " + str(content_length) + " < " + str(content_length_max))
            return False
    pd.Series(content_type)
    return True

### Function: Save Image Files

In [58]:
def saveImageFileAs(target_url, fileNamePath, filePermission = "wb"):
    data_is_good = is_downloadable(target_url,0)
    saveFileStatus = False
    if (data_is_good):
        contents = requests.get(target_url, allow_redirects=True).content
        makeDirWhenNotPresent(fileNamePath)
        with open(fileNamePath, filePermission) as fw:
            fw.write(contents)
            fw.close()
            print("Downloaded Images File at Location: \n\t" + fileNamePath)            
            saveFileStatus = True
    else:
        print("ATTENTION: Image cannot be downloaded from the following URL: \n" + \
              "\t URL: " + target_url)
        saveFileStatus = False
    
    return saveFileStatus

### Function: Download the Image for a given URL

In [59]:
def downloadContentPerImageURL(target_url_base, image_filename_with_ext, basePath = os.getcwd()):
    fileNamePath = getFullPath(image_filename_with_ext, basePath)
    saveImageFileAs(target_url_base, fileNamePath)
    
    return fileNamePath

### Function: Create image number list, An individual Image Name, Image-Name-List

1. **`makeImageNumberList`**: Create image number list
1. **`getNewImageName`**: Create an individual Image Name
1. **`makeImageNameList`**: Create Image-Name-List

In [60]:
def makeImageNumberList(total_images_number):
    # Example:
    #    total_images_number = 10
    #    image_number_list = makeImageNumberList(total_images_number)
    
    # Create list 
    image_number_list = arange(1,total_images_number+1)
    print("Total Number of Images to Download: " + str(total_images_number))
    
    return image_number_list

def zeroPad(x,string_length = 8):
    # Convert to string
    x = str(x)
    # Zero pad
    zx = x.zfill(string_length) 
    # if x is a number, zx = f'{x:03}' ==> say x = 5; zx = '005'
    # For better understanding on alternatives, use:
    #     https://stackoverflow.com/questions/339007/nicest-way-to-pad-zeroes-to-string
    #
    # NOTE: This is similar to PHP function str_pad()
    
    return zx


def getNewImageName(image_num_new, image_name_ext = "gif", image_name_length = 8):
    
    image_name_zero_padded = zeroPad(image_num_new,image_name_length)
    image_name_new = image_name_zero_padded + "." + image_name_ext
    # OLD Method: Kep for reference purpose only
    #    image_name_zero_pad = "".join([str(0)]*(image_name_length - len(str(image_num_new))))
    #    image_name_new = "".join([image_name_zero_pad, str(image_num_new), "." + image_name_ext])
    #    
    # url_parts_dict = urlparse.parse_qs(url_parts)
    # image_name = ''.join(url_parts_dict['img'])
    # image_name_parts = str.split(image_name,".")
    # image_name_ext = image_name_parts[1]
    
    return image_name_new

def makeImageNameList(manuscript_name, total_images_number, url_parts_dict, iDebugFlag = 0):
    # This function returns a tuple: (image_name_list, image_filename_list)
    # Example:
    #    manuscript_name = fileName_list[0] # fileName_list
    #    image_name_list, image_filename_list = makeImageNameList(manuscript_name, url_parts_dict,1)
    image_name = ''.join(url_parts_dict['img'])
    image_name_parts = str.split(image_name,".")
    image_name_ext = image_name_parts[1]
    image_name_length = len(image_name_parts[0])
    # Create a numeric-list for the number of images to download
    image_number_list = makeImageNumberList(total_images_number)
    image_name_list = [""]*len(image_number_list) # or, list(str(x) for x in image_number_list)
    image_filename_list = image_name_list.copy()
    filename_prefix = manuscript_name + ("" if (manuscript_name.strip() == "") else "_")
    for idx, image_number in enumerate(image_number_list):
        image_name_list[idx] = getNewImageName(image_number_list[idx], image_name_ext, image_name_length)
        image_filename_list[idx] = filename_prefix + image_name_list[idx]
        if (iDebugFlag == 1):
            print(str(idx+1) + ". " + "image_name: " + image_name_list[idx] + " | " + "image_filename: " +  image_filename_list[idx])
            
    return (image_number_list, image_name_list, image_filename_list)   

In [61]:
# Example
getNewImageName(5, 'gif', 8)

'00000005.gif'

### Function: Break URL into Relevant Parts and Combine Relevant Parts to Create New URL

1. **`getURLPartsDict`**: Breaks URL into Relevant Parts
1. **`makeURLFromParts`**: Combines Relevant Parts to Create New URL

In [62]:
def getURLPartsDict(target_url_base):
    url_parts = urlparse.unquote(target_url_base)
    url_parts_dict = urlparse.parse_qs(url_parts)
    
    return url_parts_dict

def makeURLFromParts(url_parts_dict):
    # Example:
    #    url_from_parts = makeURLFromParts(url_parts_dict)
    
    # Encode and create the url from the tuples in url_parts_dict.
    url_from_parts = urlparse.urlencode(url_parts_dict, doseq=True) 
    # Unquote the url to get intended url.
    url_from_parts = urlparse.unquote(url_from_parts)
    # Print URL
    print(url_from_parts)
    
    return url_from_parts

### Function: Make *"Base-Path"* for Saving Files

In [63]:
def getBasePathForManuscriptDownload(primary_content_title, manuscript_name, rootDirLocation = os.getcwd()):
    # Create basePath 
    subFolderList = ["Tasher_Desh", "Manuscripts", manuscript_name]
    basePath = makeSubFolderPath(subFolderList, rootDirLocation)
    print("\n Title: " + manuscript_name + "\n Manuscript Download BasePath: "  + basePath)
    
    return basePath

In [64]:
"""
# Example

target_url_base = "http://bichitra.jdvu.ac.in/utility/image.php?fpath=Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw%3D%3D&img=00000006.gif&amp;dummy=934"
url_parts_dict = getURLPartsDict(target_url_base)
url_parts_dict

"""

'\n# Example\n\ntarget_url_base = "http://bichitra.jdvu.ac.in/utility/image.php?fpath=Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw%3D%3D&img=00000006.gif&amp;dummy=934"\nurl_parts_dict = getURLPartsDict(target_url_base)\nurl_parts_dict\n\n'

### Function: Create Dummy-Key Value (Randomly) AND Get Image-URL Info:

In [65]:
def getImageURL_DummyValue(random_seed = 0):
    # from numpy import random as np_random, floor as np_floor
    if not (random_seed == 0):
        np_random.seed(random_seed)
    dummy_str = str(np_floor(np_random.random()*1000 + 1))[:-2]
    
    return dummy_str

def getImageURL_Info(target_image_url = ""):
    
    # Default Image URL
    if (target_image_url == ""):
        target_url_base = "http://bichitra.jdvu.ac.in/utility/image.php?fpath=" \
                            + "Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw" \
                            + "%3D%3D" + "&img=00000006.gif" \
                            + "&amp;dummy=934"
        target_image_url = target_url_base            
        print(target_image_url)
        
    url_parts_dict = getURLPartsDict(target_image_url)
    image_url_keys = list(url_parts_dict.keys())
    
    print(url_parts_dict)
    # Use this for better visualization: 
    #    pd.DataFrame(url_parts_dict)
    
    return (image_url_keys, url_parts_dict)

In [66]:
# Example
getImageURL_DummyValue()

'108'

In [67]:
# Example
image_url_keys, url_parts_dict = getImageURL_Info()
pd.DataFrame(url_parts_dict)

http://bichitra.jdvu.ac.in/utility/image.php?fpath=Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw%3D%3D&img=00000006.gif&amp;dummy=934
{'http://bichitra.jdvu.ac.in/utility/image.php?fpath': ['Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw=='], 'img': ['00000006.gif'], 'dummy': ['934']}


,dummy,http://bichitra.jdvu.ac.in/utility/image.php?fpath,img
0,934,Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw==,00000006.gif


In [68]:
# Example
os.path.isdir(os.path.normpath(r"C:\Users\raysu\Documents\PythonScripts\Tasher_Desh\Manuscripts"))

True

### Function: Make a Single URL from Image Name  AND Make Multiple URLs from Image Name-List

1. **`makeURLFromImageName`**: Make a Single URL from Image Name
1. **`makeImageURLs`**: Make Multiple URLs from Image Name-List

In [92]:
def makeURLFromImageName(image_name, url_parts_dict, image_file_url_key_Value, dummy_key_value = getImageURL_DummyValue()):
    # Example:
    #    url_from_parts = makeURLFromImageName(image_name, url_parts_dict)
    #
    # Make a copy of the dictionary "url_parts_dict" such that any change 
    #    to the new temporary-dictinary (url_parts_dict_temp) does not 
    #    change the original-dictionary (url_parts_dict).
    url_parts_dict_temp = url_parts_dict.copy()
    
    # Update the temp-dictionary
    url_parts_dict_temp['img'] = image_name
    url_parts_dict_temp['dummy'] = dummy_key_value
    
    # Create URL
    url_from_parts = makeURLFromParts(url_parts_dict_temp)
    
    return url_from_parts

def makeImageURLs(target_url_manuscript, image_name_list, url_parts_dict):
    # This function returns a list of URLs for the images.
    # Example:
    #    image_url_list = makeImageURLs(image_name_list, image_name_list, url_parts_dict)
    
    # Make URL Dummy-Key Value
    dummy_key_value = getImageURL_DummyValue()
    image_file_url_key_Value = getFromManuscriptURL_ImageURL_KeyValue(target_url_manuscript, 0)
    # Create URL List
    image_url_list = list(makeURLFromImageName(image_name, url_parts_dict, image_file_url_key_Value, dummy_key_value) for image_name in image_name_list)
    
    return image_url_list

### Function: Download All the Image-Files for a Given Manuscript

In [70]:
def downloadBichitraManuscriptImages(target_url_manuscript, manuscript_name = "", total_images_number = 1, basePath = os.getcwd()):
    nDigits = 3
    abs_start_time = time.time()
    print("\n " + "Downloading Image Files for Manuscript: " + manuscript_name + "\n")
    # Break the target-URL into parts (URL-parsing)
    #    url_parts_dict = getURLPartsDict(target_url_base)
    image_url_keys, url_parts_dict = getImageURL_Info()
    
    # Create three lists: **image_number_list**, **image_name_list** AND **image_filename_list**
    #    The image-URLs will be modified with image_name_list while
    #    the downloaded image-files will be saved with the names in 
    #    image_filename_list
    iDebugFlag = 0 # set this to zero to not print intermediate steps. Default: 0.
    image_number_list, image_name_list, image_filename_list = makeImageNameList(manuscript_name, total_images_number, url_parts_dict, iDebugFlag)
    image_url_list = makeImageURLs(target_url_manuscript, image_name_list, url_parts_dict)
    image_fileNamePath_list = image_filename_list.copy()
    start_time = time.time()
    for i, image_filename_with_ext in enumerate(image_filename_list):
        print("\n " + str(i+1) + ". Downloading Image File: " + image_filename_list[i])
        image_fileNamePath_list[i] = downloadContentPerImageURL(image_url_list[i], image_filename_with_ext, basePath)
        elapsed_time = time.time() - start_time
        print("\n\t Download Time: " + str(round(elapsed_time, nDigits)))
        time.sleep(1.000)
        print("\n\t Elapsed Time: " + str(round(time.time() - abs_start_time, nDigits)))
        
    total_elapsed_time = time.time() - abs_start_time
    print("\t Total Download Time: " + str(round(total_elapsed_time, nDigits)))
    
    return (image_number_list, image_name_list, image_filename_list, image_fileNamePath_list, image_url_list)

In [71]:
"""
# Example:

h = requests.head(target_url, allow_redirects=True)  
header = h.headers  
content_type = header.get('content-type')  
pd.Series(content_type)  
open('TestImage.gif', 'wb').write(contents)  
"""

"\n# Example:\n\nh = requests.head(target_url, allow_redirects=True)  \nheader = h.headers  \ncontent_type = header.get('content-type')  \npd.Series(content_type)  \nopen('TestImage.gif', 'wb').write(contents)  \n"

## Load List of Target URLs

In [72]:
target_url_list = [
     "http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=9&mname=RBVBMS_009A" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=104&mname=RBVBMS_096%28i%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=105&mname=RBVBMS_096%28ii%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=106&mname=RBVBMS_096%28iii%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=107&mname=RBVBMS_096%28iv%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=108&mname=RBVBMS_096%28v%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=119&mname=RBVBMS_101%28i%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=120&mname=RBVBMS_101%28ii%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=121&mname=RBVBMS_101%28iii%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=199&mname=RBVBMS_159" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=208&mname=RBVBMS_168" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=264&mname=RBVBMS_192%28i%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=265&mname=RBVBMS_192%28ii%29" \
    ,"http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=589&mname=BMSF_036" \
    ]

## Show Target List of URLs

In [73]:
i = 0
print("The list of target urls: \n")
for target_url in target_url_list:    
    print(str(i) + ": " + target_url)
    i = i + 1

The list of target urls: 

0: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=9&mname=RBVBMS_009A
1: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=104&mname=RBVBMS_096%28i%29
2: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=105&mname=RBVBMS_096%28ii%29
3: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=106&mname=RBVBMS_096%28iii%29
4: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=107&mname=RBVBMS_096%28iv%29
5: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=108&mname=RBVBMS_096%28v%29
6: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=119&mname=RBVBMS_101%28i%29
7: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=120&mname=RBVBMS_101%28ii%29
8: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=121&mname=RBVBMS_101%28iii%29
9: http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=199&mname=RBVBMS_159
10: http://

## Extract Filenames from the Target URLs

In [74]:
# import urllib.parse as urlparse
# Non-shallow copy of target_url_list: 
#        changes in **fileName_list** WILL NOT affect **target_url_list**
fileName_list = target_url_list.copy() 
i = 0
print("File Names: \n")
for target_url in target_url_list:
    fileName_list[i] = urlparse.unquote(target_url)
    fileName_list[i] = ''.join(urlparse.parse_qs(fileName_list[i])['mname'])
    print(str(i) + ": " + fileName_list[i] + " ")
    i = i + 1

File Names: 

0: RBVBMS_009A 
1: RBVBMS_096(i) 
2: RBVBMS_096(ii) 
3: RBVBMS_096(iii) 
4: RBVBMS_096(iv) 
5: RBVBMS_096(v) 
6: RBVBMS_101(i) 
7: RBVBMS_101(ii) 
8: RBVBMS_101(iii) 
9: RBVBMS_159 
10: RBVBMS_168 
11: RBVBMS_192(i) 
12: RBVBMS_192(ii) 
13: BMSF_036 


## Define Manuscript Downloading Function

In [75]:
def downloadManuscriptAsFile(fileName_list, target_url_list, basePath, fileExt = '.txt'):  
    nDigits = 3
    print("\n" + "File Download Progress Status: \n")
    abs_start_time = time.time()
    for i in arange(len(target_url_list)):
        start_time = time.time()
        print("\n " + str(i+1) + ". Downloading File: " + fileName_list[i])
        # Create FilePath for the given FileName
        fileNamePath = getFullPath(fileName_list[i] + fileExt, basePath)
        # Webscrape data from traget URL
        transcript_text = get_Transcript(target_url_list[i])
        # Save transcript_text to File
        fileEncoding = "utf-8"
        saveTextAsFile(transcript_text, fileNamePath, "w+", fileEncoding)
        print("\t Saved File: " + fileName_list[i] + "\n\t Web-Source: \n" + target_url_list[i])
        elapsed_time = time.time() - start_time
        print("\t Download Time: " + str(round(elapsed_time, nDigits)))
        time.sleep(1.000)
        print("\t Elapsed Time: " + str(round(time.time() - abs_start_time, nDigits)))
        
    total_elapsed_time = time.time() - abs_start_time
    print("\t Elapsed Time: " + str(round(total_elapsed_time, nDigits)))    

In [76]:
basePath = makeSubFolderPath("Tasher_Desh")
print(basePath)
downloadManuscriptAsFile(fileName_list, target_url_list, basePath)

RelativePath: C:\Users\raysu\Documents\PythonScripts\Tasher_Desh
C:\Users\raysu\Documents\PythonScripts\Tasher_Desh

File Download Progress Status: 


 1. Downloading File: RBVBMS_009A
Complete FilePath: C:\Users\raysu\Documents\PythonScripts\Tasher_Desh\RBVBMS_009A.txt
	 Saved File: RBVBMS_009A
	 Web-Source: 
http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=9&mname=RBVBMS_009A
	 Download Time: 0.598
	 Elapsed Time: 1.599

 2. Downloading File: RBVBMS_096(i)
Complete FilePath: C:\Users\raysu\Documents\PythonScripts\Tasher_Desh\RBVBMS_096(i).txt
	 Saved File: RBVBMS_096(i)
	 Web-Source: 
http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=104&mname=RBVBMS_096%28i%29
	 Download Time: 0.571
	 Elapsed Time: 3.171

 3. Downloading File: RBVBMS_096(ii)
Complete FilePath: C:\Users\raysu\Documents\PythonScripts\Tasher_Desh\RBVBMS_096(ii).txt
	 Saved File: RBVBMS_096(ii)
	 Web-Source: 
http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=105&mname=RBV

### Make Path and Read a Specific File

In [77]:
# Make Path
fileNamePath = getFullPath(fileName_list[12-1] + ".txt", basePath)

Complete FilePath: C:\Users\raysu\Documents\PythonScripts\Tasher_Desh\RBVBMS_192(i).txt


In [78]:
# Read File
file_text = readFile(fileNamePath, "utf-8", 1)




�� * *	� 
 
 
 
 *	�	�	�	�	�  	�	�	� 
 
 
 
 �	�	�	�	�	�	�  	�	�	�	�	� � 
 
 �	�	�	�	�	�  	�	�  	�	�	�	� � 
 
 
 
 *	�	�	�	�	�	�	�	�	� 
 
 < + + + >   {	�	�	�	�  	�	�	�	� }  	�	�	�  	�	�	�	�	� 
 
	�	�	�	�	�	�	�	�	�	�	�   <	�	�	�	�	�	�  	�	�	�	� >   {	�	�	�	�	�	�	�	�	�	� }  	�	�	�	�	�	�	d 
 
	�	�	�	�	�	�	�	�	�	�	�	�  	�	�	�	�  	�	�	�	�	�  	�	�	�	�	�	�	�	�  	�	�	�	�	�	� < , > {	d } 
 
 <	�	�	�	�	�  	�	�	�	�	�  	�	�	�	�	�   + + +  	�	�	�  	�	�	�	� >   {	�	�	�	�	�	�	�	�	�  	�	�	�	�	�  	�	�	�	�	�  	�	�	�  	�	�	�	� } 
 
	�	�	�	�  	�	�	�	�  	�	�	�	�  	�	�	�	�  	�	�	�	�	�  	�	�	�	�	�  	�	�	�	�	�	�	d	d 
 
 < + + +   {	�	�	�	�	� }  	�	�	�  	�	�	�	�	� >   {	�	�	�	�	�	�	�  	�	�	�	�	�  	�	�	�  	�	�	�	�	� } 
 
	�	�	�	�	�  	�	�	�	�	�  	�	�	�	�	�	�	�	�	� 
 
	�	�   <	�	�  	�	�	�	�  	�	�  	�	�	�	�	� >   {	�	�	�	�  	�	�	�	� ,  	�	�	�	�  	�	�	�	�	� , } 
 
	�	�	�	�   <	�	� >  	�	�	�	�   <	�	� >   {	�	�	�	� }  	�	�	�	�	�	�	�	d 
 
	�	�	�	�   <	�	�	�	�	�  	�	�	�	�	�	�  	�	�	�	�  	�	�	�	�	�	�   >   {	�	�	�	

### Use This to open a URL in Browser

In [79]:
import webbrowser as wb
wb.open_new_tab(target_url_list[12-1])

True

In [80]:
target_url = target_url_list[0]
# Get source html text.
source_html = requests.get(target_url).text

# Use BeautifulSoup to extract intended content.
soup = bsp(source_html, 'html.parser')
transcript_html = soup.find("div", {"id": "manuscript_img"})

print(transcript_html)

transcript_html_img = soup.find("img", {"id": "man_img_file"})

<div id="manuscript_img">
<img alt="tagore_manuscript" height="994" id="man_img_file" src="" width="1384"/></div>


### Prefix Suffix

In [81]:
prefix_str = r'../utility/image.php?fpath='
suffix_str = r'&img="+str_pad('
print(prefix_str)
print(suffix_str)


../utility/image.php?fpath=
&img="+str_pad(


In [82]:
image_url_parts = 

SyntaxError: invalid syntax (<ipython-input-82-f050e0bb648a>, line 1)

In [83]:
target_url = target_url_list[0]
# Get source html text.
source_html = requests.get(target_url)
print(source_html.text)

# Use BeautifulSoup to extract intended content.
soup_head = bsp(source_html.text, 'html.parser').find("head")
soup_head_js = soup_head.findAll("script", {"type": "text/javascript"})

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />

<title>RBVBMS_009A :: Manuscript of Rabindranath Tagore :: Bichitra Tagore Electronic Hypertext Project :: School of Cultural Texts and Records</title>
<link href="../css/tags_bn.css" type="text/css" rel="stylesheet" />
<link href="../css/text_en.css" type="text/css" rel="stylesheet" />
<link href="css/viewer.css" type="text/css" rel="stylesheet" />
<link href="../css/sunny/jquery-ui-1.10.0.custom.min.css" type="text/css" rel="stylesheet" />

<script src="../js/jquery-1.8.2.min.js" type="text/javascript"></script>
<script src="../js/jquery-ui-1.10.0.custom.min.js" type="text/javascript"></script>
<script src="../js/jquery.easing.1.3.js" type="text/javascript"></script>
<script src="../js/jQueryRotateCompressed.2.2.js" type="text/javascript"></scr

In [84]:
pd.Series(soup_head_js)

0    <script src="../js/jquery-1.8.2.min.js" type="...
1    <script src="../js/jquery-ui-1.10.0.custom.min...
2    <script src="../js/jquery.easing.1.3.js" type=...
3    <script src="../js/jQueryRotateCompressed.2.2....
4    <script src="../js/jquery.mousewheel.min.js" t...
5    <script type="text/javascript">
<!--
$im= jQ...
6    <script src="js/manuscript_view.js" type="text...
7    <script type="text/javascript">
<!--
functio...
dtype: object

In [85]:
print(transcript_html_img)

<img alt="tagore_manuscript" height="994" id="man_img_file" src="" width="1384"/>


In [86]:
print(source_html)

<Response [200]>


In [467]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   RBVBMS_009A :: Manuscript of Rabindranath Tagore :: Bichitra Tagore Electronic Hypertext Project :: School of Cultural Texts and Records
  </title>
  <link href="../css/tags_bn.css" rel="stylesheet" type="text/css"/>
  <link href="../css/text_en.css" rel="stylesheet" type="text/css"/>
  <link href="css/viewer.css" rel="stylesheet" type="text/css"/>
  <link href="../css/sunny/jquery-ui-1.10.0.custom.min.css" rel="stylesheet" type="text/css"/>
  <script src="../js/jquery-1.8.2.min.js" type="text/javascript">
  </script>
  <script src="../js/jquery-ui-1.10.0.custom.min.js" type="text/javascript">
  </script>
  <script src="../js/jquery.easing.1.3.js" type="text/javascript">
  </script>
  <script src="../js/jQueryRotateCompressed.2.2.js

In [480]:
f'{8:03}'

'008'

## Prepare Data for Manuscript Image-File Download

### Set Parameters to Start Download

In [40]:
def setParams_Download_Manuscript_Image_Files(total_image_pages, target_url_list, fileName_list, primary_content_title = "Tasher_Desh", download_index = 0):
    # Primary Content Title
    # primary_content_title = "Tasher_Desh" 

    # Download Index: Pick the index with Manuscript you want 
    #                 to download from the LIST: 
    #                     manuscript_image_url_list
    # download_index = 13 # Choose between: 0 - 13

    # Manuscript Name
    manuscript_name = fileName_list[download_index] # fileName_list

    # Manuscript URL
    target_url_manuscript = target_url_list[download_index]

    # Target URL for any of the Image-File(s)
    # target_url_base = manuscript_image_url_list[download_index][1]

    # Total Number of Pages/Images available on Bichitra for 
    # the Manuscript (Click on the "Toolbox": Top-Right)
    #total_images_number = manuscript_image_url_list[download_index][0]
    total_images_number = total_image_pages[download_index]

    # Process input: Replace blanks by underscores from "primary_content_title"
    primary_content_title = primary_content_title.replace(" ", "_") 

    # Print the input variables
    print("\n Manuscript belongs to Title: " + primary_content_title)
    print("\n Manuscript Name: " + manuscript_name)
    print("\n Manuscript URL: \n" + target_url_manuscript)
    print("\n Total Number of Image-Files: " + str(total_images_number))

    # Make basePath
    basePath = getBasePathForManuscriptDownload(primary_content_title, manuscript_name)
    
    return (target_url_manuscript, manuscript_name, total_images_number, basePath)

## Prepare Variables for Manuscript Image-Files Download

#### What you need:
1. **`primary_content_title`**: This is the title. Example: *Tasher Desh*
1. **`download_index`**: Index of the Manuscript in the List: **manuscript_image_url_list**
1. **`manuscript_name`**: Picked from the LIST: **fileName_list**
1. **`target_url_base`**: Picked from the List: **manuscript_image_url_list**
1. **`total_images_number`**: Picked from the List: **manuscript_image_url_list**

In [42]:
total_image_pages = [98,35,29,29,28,29,29,29,29,291,86,63,2,4]
for i,target_url in enumerate(target_url_list):
    print(str(i) + "\t" + target_url)

0	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=9&mname=RBVBMS_009A
1	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=104&mname=RBVBMS_096%28i%29
2	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=105&mname=RBVBMS_096%28ii%29
3	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=106&mname=RBVBMS_096%28iii%29
4	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=107&mname=RBVBMS_096%28iv%29
5	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=108&mname=RBVBMS_096%28v%29
6	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=119&mname=RBVBMS_101%28i%29
7	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=120&mname=RBVBMS_101%28ii%29
8	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=121&mname=RBVBMS_101%28iii%29
9	http://bichitra.jdvu.ac.in/manuscript/manuscript_viewer.php?manid=199&mname=RBVBMS_159
10	http://bichitra.jdvu.ac.in/manuscript/manuscr

## Run Image-Downloader for a Given Manuscript

### Profile Function: 

In [41]:
"""
# This did not work from the notebook. Test later.

from pycallgraph import PyCallGraph
from pycallgraph.output import GraphvizOutput

with PyCallGraph(output=GraphvizOutput()):
    image_number_list, image_name_list, image_filename_list, image_fileNamePath_list, image_url_list = \
    downloadBichitraManuscriptImages(target_url_manuscript, manuscript_name, total_images_number, basePath)

"""


'\n# This did not work from the notebook. Test later.\n\nfrom pycallgraph import PyCallGraph\nfrom pycallgraph.output import GraphvizOutput\n\nwith PyCallGraph(output=GraphvizOutput()):\n    image_number_list, image_name_list, image_filename_list, image_fileNamePath_list, image_url_list =     downloadBichitraManuscriptImages(target_url_manuscript, manuscript_name, total_images_number, basePath)\n\n'

In [697]:
download_index = 12
manuscript_name = "Tasher_Desh"
# Get setup parameters for downloading manuscript-images
target_url_manuscript, manuscript_name, total_images_number, basePath = \
    setParams_Download_Manuscript_Image_Files(total_image_pages, \
                                              target_url_list, \
                                              fileName_list, \
                                              manuscript_name, \
                                              download_index)

# Begin manuscript download    
%prun image_number_list, image_name_list, image_filename_list, image_fileNamePath_list, image_url_list = \
    downloadBichitraManuscriptImages(target_url_manuscript, manuscript_name, total_images_number, basePath)
    



http://bichitra.jdvu.ac.in/utility/image.php?fpath=Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw%3D%3D&img=00000006.gif&amp;dummy=934
{'http://bichitra.jdvu.ac.in/utility/image.php?fpath': ['Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw=='], 'img': ['00000006.gif'], 'dummy': ['934']}
Total Number of Images to Download: 4
http://bichitra.jdvu.ac.in/utility/image.php?fpath=Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw==&img=00000001.gif&dummy=964
http://bichitra.jdvu.ac.in/utility/image.php?fpath=Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw==&img=00000002.gif&dummy=964
http://bichitra.jdvu.ac.in/utility/image.php?fpath=Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw==&img=00000003.gif&dummy=964
http://bichitra.jdvu.ac.in/utility/image.php?fpath=Y29udGVudC9tYW51c2NyaXB0L1JCVkJNU18wMDlBLw==&img=00000004.gif&dummy=964

 1. Downloading Image File: BMSF_036_00000001.gif
Complete FilePath: C:\Users\raysu\Documents\PythonScripts\Tasher_Desh\Manuscripts\BMSF_036\BMSF_036_00000001.gif
content-type: image/gif
D

## Rotate Upside-Down Images for Manuscript: *RBVBMS_192(i)*

The files were downloaded using the python-script above. For **manuscript-RBVBMS_192(i)** the image-files clearly had three problems:

1. **Incorrect page order**.
2. **Upside-down pages**: 51 out of 63 (81%) pages are oriented upside-down.
3. **No meaningful transcription exists**.

In an attempt to address the problems identified above, the files were first **copied to** a subfolder (*Corrected_Images*) and then renamed using [Bulk Renaming Utility](http://www.bulkrenameutility.co.uk/Download.php). The renaming was done to achieve the following tasks:

1. **Establish correct page order**: A prefix-scheme was used to correctly label the images to establish the correct page-order.
1. **Fix upside-down pages issue**: 
    1. Label pages that were scanned upside-down (pages: 2 - 52 | Total: 63)
    1. Upside-down pages (*images*) were moved to another subfolder (*ImagesToRotate*) under folder: *Corrected_Images*
    1. Run a python-script to bulk-rotate all the images in the subfolder: *ImagesToRotate*
1. **Recombine all *corrected* images** to create the correct set of images for **manuscript-RBVBMS_192(i)**.